In [1]:
import numpy as np
import matplotlib.pyplot as pl
import matplotlib.style as mplstyle
import seaborn as sns

import sys
sys.path.insert(1, '../')
import src.core.gaussian_beam_propagation as gbp
from src.core.gaussian_beam_propagation import FocusedGaussianBeam
from src.core import intersect

mplstyle.use('seaborn')
sns.set_style("darkgrid", {'axes.grid' : False})

from matplotlib.colors import ListedColormap
cmap = ListedColormap(sns.cubehelix_palette(100, start=.5, rot=-.75, reverse=True))

/var/folders/k7/yq0x0lnn2ns__6748g658mxm0000gn/T/ipykernel_45089/714872020.py:12: MatplotlibDeprecationWarning: The seaborn styles shipped by Matplotlib are deprecated since 3.6, as they no longer correspond to the styles shipped by seaborn. However, they will remain available as 'seaborn-v0_8-<style>'. Alternatively, directly use the seaborn API instead.
  mplstyle.use('seaborn')


In [23]:
datadir = "../data/calibration_water"

In [13]:
print (datadir)

../data/calibration_water/


In [14]:
def filename(code, num_photons, num_steps, scat_deg, g, trial, mod=None):
    names = [f'ray', f'out', f'med', f'his', f'medprop']
    desc = f'N{num_photons}s{num_steps}R{int(scat_deg)}g{int(g*10)}t{trial}'
    if mod is not None:
        desc += mod
    return names[code] + desc + f'.npy'

In [15]:
def load_id(scattering_degree, g, trials):
    subfoldir = '' # f'r{scattering_degree}'
    fullpath = os.path.join(datadir, subfoldir)
    rays = np.load(os.path.join(fullpath, f'rayR{int(scattering_degree)}g{int(g*10)}t{trial}.npy'))
#     rays = np.load('r2' + f'rayR{int(scattering_degree)}g{int(g*10)}t{trial}.npy')
    scattered_out = np.load(os.path.join(fullpath, f'outR{int(scattering_degree)}g{int(g*10)}t{trial}.npy'))
    scatterer_centers = np.load(os.path.join(fullpath, f'medR{int(scattering_degree)}g{int(g*10)}t{trial}.npy'))
    scatter_history = np.load(os.path.join(fullpath, f'hisR{int(scattering_degree)}g{int(g*10)}t{trial}.npy'))
    scat_props = np.load(os.path.join(fullpath, f'medpropR{int(scattering_degree)}g{int(g*10)}t{trial}.npy'))
    
    rays = np.concatenate(rays, axis=1)
    scattered_out = np.concatenate(scattered_out.reshape(10, 1000, 100), axis=0)
    scatter_history = np.concatenate(scatter_history.reshape(10, 1000, 100), axis=0)
    
    return rays, scattered_out, scatterer_centers, scatter_history, scat_props

In [16]:
def load_and_join_trials(code, num_trials, num_photons, num_steps, scattering_degree, g, mod=None):
    subfoldir = '' # f'r{scattering_degree}'
    fullpath = os.path.join(datadir, subfoldir)
    file = filename(code, num_photons, num_steps, scattering_degree, g, trial=0)
    t0 = np.load(os.path.join(fullpath, file))
    arr = []

    for trial in range(num_trials):
        temp = np.load(os.path.join(fullpath, filename(code, num_photons, num_steps, scattering_degree, g, trial, mod)))
#         if code == 0:
#             temp = temp.reshape(10, 3, temp.shape[-2], temp.shape[-1]//10)
#             temp = np.concatenate(temp, axis=1)
#         elif (code == 1) or (code == 3):
#             temp = temp.reshape(10, temp.shape[-2], temp.shape[-1]//10)
#             temp = np.concatenate(temp, axis=0)
        arr.append(temp)
#         arr[trial] = np.load(subfoldir + filename(code, scattering_degree, g, trial))
        
    return np.array(arr)

In [17]:
def rays_to_contour(rays, axial_range=20, resolution=201):
    num_steps = rays.shape[2]
    axial_planes = np.linspace(-axial_range, axial_range, resolution)
    intersections_per_plane = [[],[]]
    
    for p in range(len(axial_planes)):
        for step in range(num_steps-1):
            # check if rays intersect this plane at all
            plane = axial_planes[p]
            intersects_plane = (rays[2, :, step+1] > plane) & (rays[2, :, step] < plane)
            # calculate points of intersection for this plane
            intersections = intersect.ray_xyplane(plane, rays[:, intersects_plane, step], rays[:, intersects_plane, step+1])
            intersections_per_plane[0].extend(intersections[0])
            intersections_per_plane[1].extend(intersections[2])
            
    return intersections_per_plane, axial_planes

In [18]:
def linfoot(signal_expt, signal_theo):
    '''
    C: relative structural content
    F: fidelity
    Q: correlation quality
    '''
    # normalize
    sig_expt_sum = np.sum(signal_expt)
    sig_theo_sum = np.sum(signal_theo)
    if sig_expt_sum != 0:
        signal_expt /= sig_expt_sum
    if sig_theo_sum != 0:
        signal_theo /= sig_theo_sum
    C = np.sum(signal_expt**2) / np.sum(signal_theo**2)
    F = 1 - np.sum((signal_expt - signal_theo)**2)/np.sum(signal_theo**2)
    Q = np.sum(signal_expt*signal_theo) / np.sum(signal_theo**2)

    return C, F, Q

In [19]:
def plots2(X, Z, intensity, alpha):
    extent = (-15, 15, -15//2, 15//2)
    print(extent)
    # X for x axis, Z for y axis
    fig = pl.figure()

    ax1 = pl.subplot2grid((3, 5), (0, 0), colspan=4)
    ax2 = pl.subplot2grid((3, 5), (1, 0), colspan=4, rowspan=2)
    ax3 = pl.subplot2grid((3, 5), (1, 4), rowspan=2)

    ax1.plot(X[0], intensity[intensity.shape[0]//2, :])
    ax1.set_xlim(extent[0], extent[1])
    ax2.contour(intensity, levels=np.logspace(-4,1, 20), cmap=pl.cm.gray, linewidths=0.5, extent=extent)
#     ax2.contour(intensity, levels=[np.amax(intensity)/np.e**2], cmap=pl.cm.gray, linewidths=1.0, extent=extent)
    ax2.imshow(intensity, cmap=cmap, extent=extent)
    ax3.plot(intensity[:, intensity.shape[1]//2], Z[:, 0])
    ax3.set_ylim(extent[2], extent[3])
    
    pl.tight_layout()


In [20]:
def get_theo_dists(beam):
    u = np.linspace(-30, 30, 200, dtype=np.float64)
    v = np.linspace(-15, 15, 100, dtype=np.float64)
    
    V, U, field = beam.debye_approx_field(v, u)
    intensity = np.array(np.abs(field)**2, dtype=np.float64)
    intensity = intensity / np.amax(intensity)
    
    axial_profile = intensity[intensity.shape[0]//2, :]
    transverse_profile = intensity[:, intensity.shape[1]//2]
    
    return u, v, intensity, transverse_profile, axial_profile

In [22]:
R = []
num_trials = 5
g = 0.2
num_photons = 30000
num_steps = 100
step_param_arr = np.logspace(-18.4, -18, 10)
# step_param_arr = [8.2e-18]
scat_degree = 0
for s in range(len(step_param_arr)):
    step_param = step_param_arr[s]
    if scat_degree == 0:
        g = 0
    else: 
        g = 0.2
    rays = load_and_join_trials(0, num_trials=num_trials, num_steps=step_param, num_photons=num_photons,
                                scattering_degree=scat_degree, g=g)
    R.append(rays)
    
transverse_intensities = []
axial_intensities = []
axial_intensities_std = []
hists = []
transverse_edges = []
axial_edges = []

h_ave = []
h_std = []

for s in range(len(R)): 
    print(s)
    rays = R[s]
    for trial in range(5):
#         print(rays[trial].shape)
        intersections_per_plane, axial_planes = rays_to_contour(rays[trial])
        intersections_per_plane = np.array(intersections_per_plane)
#         print(intersections_per_plane.shape)
        h, xedges, yedges = np.histogram2d(intersections_per_plane[0], intersections_per_plane[1], bins=[axial_planes, axial_planes])
        hists.append(h)
    histsarr = np.array(hists)
    h_ave.append(np.average(histsarr, axis=0))
    h_std.append(np.std(histsarr, axis=0))
    del histsarr

#     print(h_ave[scat_degree].shape)
    transverse_intensities.append(h_ave[s][:, h_ave[s].shape[1]//2])
    axial_intensities.append(h_ave[s][h_ave[s].shape[1]//2, :])
    axial_intensities_std.append(h_std[s][h_std[s].shape[1]//2, :])
    hists.append(h)
    axial_edges.append(xedges)
    transverse_edges.append(yedges)

FileNotFoundError: [Errno 2] No such file or directory: '../data/calibration_water/rayN30000s3.9810717055349853e-19R0g0t0.npy'

In [ ]:
np.save('transverse_edges.npy', np.array(transverse_edges))

In [25]:
transverse_intensities = np.load(f'{datadir}/transverse_intensities.npy')
axial_intensities = np.load(f'{datadir}/axial_intensities.npy')
axial_intensities_std = np.load(f'{datadir}/axial_intensities_std.npy')
h_ave = np.load(f'{datadir}/h_ave.npy')
h_std = np.load(f'{datadir}/h_std.npy')
axial_edges = np.load(f'{datadir}/axial_edges.npy')
transverse_edges = np.load(f'{datadir}/transverse_edges.npy')

FileNotFoundError: [Errno 2] No such file or directory: '../data/calibration_water/h_ave.npy'

In [ ]:
R_nocurve = []
num_trials = 5
g = 0.2
num_photons = 30000
num_steps = 100
# step_param_arr = np.logspace(-18.5, -17.5, 10)
# step_param_arr = [8.2e-18]
scat_degree = 0
for s in range(len(step_param_arr)):
    step_param = step_param_arr[s]
    if scat_degree == 0:
        g = 0
    else: 
        g = 0.2
    rays = load_and_join_trials(0, num_trials, num_photons, step_param_arr[0], scat_degree, g, mod=f'-nocurve')
    R_nocurve.append(rays)

In [ ]:
beam = FocusedGaussianBeam(NA=0.4, n=1.33, z_f=0, trunc_coeff=4)
x = (transverse_edges[scat_degree][1:] + transverse_edges[scat_degree][1:])/2
z = (axial_edges[scat_degree][1:] + axial_edges[scat_degree][:-1])/2

u, v, intensity, transverse_profile, axial_profile = get_theo_dists(beam)

In [ ]:
mcbeam = gbp.FocusedGaussianBeamMC(num_photons=1.0E3, num_iterations=100, curvature_correction=False, axial_resolution=200, beam_dist='gaussian', num_steps=100, NA=0.4, n=1.33, z_f=0, trunc_coeff=4)

In [ ]:
transverse_intensities_nc = []
axial_intensities_nc = []
axial_intensities_std_nc = []
hists_nc = []
transverse_edges_nc = []
axial_edges_nc = []

h_ave_nc = []
h_std_nc = []

for s in range(len(R_nocurve)): 
    rays = R_nocurve[s]
    for trial in range(5):
        h, trans_edges, ax_edges = mcbeam.rays_to_intensity(rays[trial],
                                                            trans_img_range=[transverse_edges[0][0], transverse_edges[0][-1]],
                                                            axial_img_range=[axial_edges[0][0], axial_edges[0][-1]],
                                                            axial_resolution=201)
        hists_nc.append(h)
    histsarr = np.array(hists_nc)
    h_ave_nc.append(np.average(histsarr, axis=0))
    h_std_nc.append(np.std(histsarr, axis=0))
    del histsarr

#     print(h_ave[scat_degree].shape)
    transverse_intensities_nc.append(h_ave_nc[s][:, h_ave_nc[s].shape[1]//2])
    axial_intensities_nc.append(h_ave_nc[s][h_ave_nc[s].shape[1]//2, :])
    axial_intensities_std_nc.append(h_ave_nc[s][h_ave_nc[s].shape[1]//2, :])
    axial_edges_nc.append(ax_edges)
    transverse_edges_nc.append(trans_edges)

In [ ]:
axial_edges_nc[0].shape

In [ ]:
X, Z = np.meshgrid(axial_edges_nc[0], transverse_edges_nc[0])
plots2(X, Z, h_ave_nc[0], alpha=4)

In [ ]:
x = (transverse_edges[0][1:] + transverse_edges[0][:-1] ) / 2
z = (axial_edges[0][1:] + axial_edges[0][:-1] ) / 2
X, Z = np.meshgrid(x, z)
for s in range(len(step_param_arr)):
    plots2(X, Z, h_ave[s], alpha=4)

In [ ]:
beam.z_R, beam.w0, beam.f, beam.wavelength, beam.aperture, beam.focal_tolerance

In [ ]:
U, V = np.meshgrid(u, v)
X, Z = beam.change_coords_reverse(V, U)

In [ ]:
plots2(U, V, intensity, alpha=4)

In [ ]:
# pl.figure()
x = (transverse_edges[0][1:] + transverse_edges[0][:-1] ) / 2
z = (axial_edges[0][1:] + axial_edges[0][:-1] ) / 2

axint_theo = beam.axial_intensity(z)
axint_theo /= np.amax(axint_theo)
axint_theo[axint_theo==0] = 1


znc = (axial_edges_nc[0][1:] + axial_edges_nc[0][:-1] ) / 2
print(znc.shape)
axint_theonc = beam.axial_intensity(znc)
axint_theonc /= np.amax(axint_theonc)
axint_theonc[axint_theonc==0] = 1


ztrunc1 = np.abs(z) <= 15 
ztrunc2 = np.abs(znc) <= 15

xt, zt = beam.change_coords_reverse(v, u)


for s in range(len(R)):
    pl.figure()
    
    ax_int_normed = axial_intensities[s] / np.amax(axial_intensities[s])
    ax_std_normed = axial_intensities_std[s] / np.amax(axial_intensities_std[s])
#     print(f'cross correlation: {cross_correlate_1d(ax_int_normed, axint_theo)}')
    C, F, Q = linfoot(ax_int_normed[ztrunc1], axint_theo[ztrunc1])
    pl.plot(z[ztrunc1], ax_int_normed[ztrunc1], label=f'SMC: C = {C:.2f}, F = {F:.2f}, Q = {Q:.2f}')
    # pl.fill_between(x, ax_int_normed-ax_std_normed, ax_int_normed+ax_std_normed, alpha=0.4)
    
    
    ax_int_normed_nc = axial_intensities_nc[0] / np.amax(axial_intensities_nc[0])
    C, F, Q = linfoot(ax_int_normed_nc[ztrunc2], axint_theonc[ztrunc2])
    pl.plot(znc[ztrunc2], ax_int_normed_nc[ztrunc2], label=f'CMC: C = {C:.2f}, F = {F:.2f}, Q = {Q:.2f}')
    
    pl.plot(z, axint_theo/np.amax(axint_theo), label=f'SDT - Tanaka et al.')
    pl.plot(zt, axial_profile, label=f'SDT - Horvath & Bor')
    pl.xlim(-15, 15)
    pl.xlabel(r'$z$ ($\mu$m)')
    pl.ylabel('Normalized intensity')
    pl.title(f'$\epsilon$ = {step_param_arr[s]:.2E}')
    pl.axvline(x=-0.5, color='gray')
    pl.axvline(x=0.5, color='gray')
    
    print()
    print(f's = {step_param_arr[s]:.2E}')
    print(f'Rayleigh length MC: {z[np.argmin(np.abs(ax_int_normed-0.5))]}')
    print(f'Rayleigh length SDT2: {zt[np.argmin(np.abs(axial_profile-0.5))]}')
    print(f'Rayleigh length SDT: {z[np.argmin(np.abs(axint_theo/np.amax(axint_theo)-0.5))]}')
    print(f'{C}, {F}, {Q}')
    # pl.ylim(0, 1)
    lgd = pl.legend(loc='center left', bbox_to_anchor=(1, 0.5))
#     pl.gca().add_artist(legend1)

    arrow = -10.8, 0.958, 3, 0
    pl.arrow(*arrow, shape='full', lw=0.5, length_includes_head=False, head_length=0.3, head_width=0.03, color='k')
    pl.text(-14.5, 0.95, 'beam propagation \ndirection')
    
    pl.savefig(f'axialint-{s}.png', dpi=300, bbox_extra_artists=(lgd,), bbox_inches='tight')

In [ ]:
# pl.figure()
x = (transverse_edges[0][1:] + transverse_edges[0][:-1] ) / 2
z = (axial_edges[0][1:] + axial_edges[0][:-1] ) / 2
vt, ut = beam.change_coords(x, z)
trint_theo = beam.transverse_intensity(x, 0)
x2 = np.linspace(x[0], x[-1], 100)
x3, z3 = beam.change_coords_reverse(v, u)

for s in range(len(R)):
    pl.figure()
#     C, F, Q = linfoot(axial_intensities[s], axint_theo)
    tr_int_normed = transverse_intensities[s] / np.amax(transverse_intensities[s])
#     print(f'cross correlation: {cross_correlate_1d(ax_int_normed, axint_theo)}')
    pl.plot(x, tr_int_normed, label=f'MC')
#     pl.plot(x, trint_theo/np.amax(trint_theo), label=f'SDT')
    pl.plot(x3, transverse_profile, label=f'SDT - Horvath & Bor')
    pl.xlim(-2, 2)
    pl.xlabel(r'$x$ ($\mu$m)')
    pl.ylabel('Normalized intensity')
    pl.title(f's = {step_param_arr[s]:.2E}')
    
    # pl.ylim(0, 1)
    pl.legend(loc='best')
    pl.savefig(f'transverseint-{s}.png', dpi=300)
    
    print(f'Beam waist MC: {x[np.argmin(np.abs(tr_int_normed - (1/np.e**2)))] - np.mean(x)}')
    print(f'Beam waist SDT: {x3[np.argmin(np.abs(transverse_profile - (1/np.e**2)))] - np.mean(x3)}')
#     print(f'Beam waist SDT: {x[np.argmin(np.abs(trint_theo/np.amax(trint_theo) - (1/np.e**2)))]}')
    print()

In [ ]:
pl.plot(x3, np.abs(transverse_profile - (1/np.e**2)))
pl.plot(x, np.abs(tr_int_normed - (1/np.e**2)))
pl.xlim(-2, 2)

In [ ]:
C_list = []
F_list = []
Q_list = []
for s in range(len(R)):
#     pl.figure()
    I = h_ave[s].T / np.amax(h_ave[s])
    X, Z = np.meshgrid(transverse_edges[scat_degree][:-1], axial_edges[scat_degree][:-1])
    cropped = (X < 15) & (Z< 15)
    crop_x, crop_z = np.where(cropped == 1)
    print(crop_x, crop_z)
    
#     pl.contour(X, Z, I, levels=np.logspace(-6, 0, 8), cmap=pl.cm.gray, origin='lower', linewidths=0.5)
#     pl.title(f'R = {(scat_degree) * 2}')
#     pl.xlabel(r'$z$ ($\mu$m)')
#     pl.ylabel(r'$x$ ($\mu$m)')
    C, F, Q = linfoot(I, intensity[:, intensity.shape[1]//2])
    print(C, F, Q)
    print(2*Q - (C + F))
    C_list.append(C)
    F_list.append(F)
    Q_list.append(Q)

        
#     pl.savefig(f'intensitydist-R{scat_degree * 2}.png', dpi=300)

In [ ]:
beam.w0

In [ ]:
pl.figure()
pl.semilogx(step_param_arr, C_list, '.')
pl.figure()
pl.semilogx(step_param_arr, F_list, '.')
pl.figure()
pl.semilogx(step_param_arr, Q_list, '.')

step_param_arr[np.argmax(np.array(C_list)[1:])]

In [ ]:
mean_free_path = []
path_length = []

for scat_degree in range(len(R)): 
    free_paths = FREE_PATHS[scat_degree]
    start_step = START_STEP[scat_degree]
    stop_step = STOP_STEP[scat_degree]
    mean_free_path.append([])
    path_length.append([])
    
    for trial in range(5):
        paths_in_medium = []
        ballistic = 0
        for photon in range(free_paths.shape[1]):
            start = start_step[trial, photon]
            stop = stop_step[trial, photon]
            if start==stop:
                ballistic += 1
            paths_in_medium.extend(free_paths[trial, photon, start:stop-2])
        mean_free_path[scat_degree].append(np.average(np.array(paths_in_medium)))
        path_length[scat_degree].append(np.sum(np.array(paths_in_medium)))
        print(f'number of ballistic photons: {ballistic}')

mean_free_path = np.array(mean_free_path)
path_length = np.array(path_length)
print()
print(mean_free_path)
print()
print(path_length)

#     print(f'R={scat_degree*2} \n Mean free path: {average_mfp} \n Average number of times scattered: {average_num_times_scattered} \n')

In [ ]:
step_param_arr[8]

In [ ]:
ax = pl.subplot()
# ax.set_xscale("log", nonposx='clip')
# ax.set_yscale("log", nonposy='clip')
x = 1 / np.arange(2, 5, 2)
y = np.average(mean_free_path, axis=1)
yerr = np.std(mean_free_path, axis=1)
pl.errorbar(x, y, yerr=yerr, fmt='o')
# pl.plot(x, y, 'o')
pl.xlabel(r'mean free path from Degree of scattering ($d_s = h / R$)')
pl.ylabel('Calculated path ($d_s\'$)')
# pl.title('Calculated degree of scattering')
## do for higher degrees of scattering



In [ ]:
mean_free_path

In [ ]:
pl.plot(np.arange(2,9,2), np.average(mean_free_path[1:], axis=1), 'o')
pl.xlabel(r'Degree of scattering $R = h/d_s$')
pl.ylabel(r'Mean free path ($\mu s$)')